In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, MaxPooling1D, Conv1D, GlobalMaxPooling1D, Dropout, LSTM, GRU
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

2022-11-17 09:56:58.980460: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('./nlp/USA_Today.csv')

In [3]:
df.head()

,date,timestamp,title,level2,level3
0,2012/10/01,1349064000000,"Catchings, January help Fever even series with...",sports,wnba
1,2012/10/01,1349064000000,Kyle Busch rants on radio after his Toyota fal...,sports,nascar
2,2012/10/01,1349064000000,Schwarzenegger says 'You can't run from your m...,life,books
3,2012/10/01,1349064000000,Ryder Cup Day 3,sports,golf
4,2012/10/01,1349064000000,Regular officials blow another big call agains...,gameon,NaN


In [4]:
num_words = 10000
max_news_len = 29
nb_classes_1 = 69
nb_classes_2 = 264


In [5]:
df = df.fillna(0)
news = df['title']
tags_1 = df['level2'] # 69
tags_2 = df['level3'] # 264


In [6]:
tokinezer_text = Tokenizer(num_words=num_words)
tokinezer_text.fit_on_texts(news)

In [7]:
tokinezer_text.word_index

{'to': 1,
 'in': 2,
 'for': 3,
 'the': 4,
 'of': 5,
 'on': 6,
 'a': 7,
 'with': 8,
 'at': 9,
 'and': 10,
 'new': 11,
 'is': 12,
 'after': 13,
 'from': 14,
 'no': 15,
 'over': 16,
 'as': 17,
 'win': 18,
 '2': 19,
 'state': 20,
 'trump': 21,
 '3': 22,
 '1': 23,
 'by': 24,
 '5': 25,
 'game': 26,
 'out': 27,
 'top': 28,
 'up': 29,
 '4': 30,
 'beat': 31,
 'be': 32,
 'past': 33,
 'will': 34,
 'off': 35,
 'best': 36,
 'says': 37,
 'year': 38,
 'you': 39,
 'first': 40,
 'has': 41,
 'nfl': 42,
 'more': 43,
 'are': 44,
 'beats': 45,
 'day': 46,
 'not': 47,
 'this': 48,
 's': 49,
 'back': 50,
 'lead': 51,
 'how': 52,
 'season': 53,
 'about': 54,
 'u': 55,
 'your': 56,
 'big': 57,
 'world': 58,
 '10': 59,
 'coach': 60,
 'leads': 61,
 'his': 62,
 '6': 63,
 'wins': 64,
 'scores': 65,
 'week': 66,
 'get': 67,
 'what': 68,
 'cup': 69,
 'open': 70,
 'vs': 71,
 'man': 72,
 '7': 73,
 'it': 74,
 'gets': 75,
 'all': 76,
 '0': 77,
 'us': 78,
 'time': 79,
 'home': 80,
 'can': 81,
 'against': 82,
 'into': 83,

In [8]:
tokinezer_tags_1 = Tokenizer(num_words=69)
tokinezer_tags_1.fit_on_texts(tags_1)

In [9]:
tokinezer_tags_1.word_index

{'sports': 1,
 'news': 2,
 'life': 3,
 'money': 4,
 'tech': 5,
 'travel': 6,
 'opinion': 7,
 'entertainment': 8,
 'weather': 9,
 'college': 10,
 'gameon': 11,
 'happyeverafter': 12,
 'theoval': 13,
 'todayinthesky': 14,
 'nletter': 15,
 'popcandy': 16,
 'cruiselog': 17,
 'onpolitics': 18,
 'driveon': 19,
 'experience': 20,
 'dispatches': 21,
 'your': 22,
 'take': 23,
 'nation': 24,
 'idolchatter': 25,
 'cybertruth': 26,
 'hotelcheckin': 27,
 'interactives': 28,
 'ondeadline': 29,
 'reality': 30,
 'technologylive': 31,
 'lightpost': 32,
 'virtual': 33,
 'community': 34,
 'hub': 35,
 'entertainthis': 36,
 'augmented': 37,
 'home': 38,
 'graphics': 39,
 'aerial': 40,
 'journalism': 41,
 'some': 42,
 'bistro': 43,
 'gamehunters': 44,
 'grateful': 45,
 'world': 46,
 'ncaaw': 47,
 'mlb': 48,
 'ncaaf': 49,
 'system': 50,
 'movies': 51,
 'fantasy': 52,
 'cars': 53,
 'cycling': 54,
 'a5c9ad5f': 55,
 'fc8d': 56,
 '49ff': 57,
 'ba16': 58,
 '6279e1e40e63': 59,
 '25d6c598': 60,
 '048d': 61,
 '487b'

In [170]:
rezi = tokinezer_tags_1.word_index
print(rezi)

{'sports': 1, 'news': 2, 'life': 3, 'money': 4, 'tech': 5, 'travel': 6, 'opinion': 7, 'entertainment': 8, 'weather': 9, 'college': 10, 'gameon': 11, 'happyeverafter': 12, 'theoval': 13, 'todayinthesky': 14, 'nletter': 15, 'popcandy': 16, 'cruiselog': 17, 'onpolitics': 18, 'driveon': 19, 'experience': 20, 'dispatches': 21, 'your': 22, 'take': 23, 'nation': 24, 'idolchatter': 25, 'cybertruth': 26, 'hotelcheckin': 27, 'interactives': 28, 'ondeadline': 29, 'reality': 30, 'technologylive': 31, 'lightpost': 32, 'virtual': 33, 'community': 34, 'hub': 35, 'entertainthis': 36, 'augmented': 37, 'home': 38, 'graphics': 39, 'aerial': 40, 'journalism': 41, 'some': 42, 'bistro': 43, 'gamehunters': 44, 'grateful': 45, 'world': 46, 'ncaaw': 47, 'mlb': 48, 'ncaaf': 49, 'system': 50, 'movies': 51, 'fantasy': 52, 'cars': 53, 'cycling': 54, 'a5c9ad5f': 55, 'fc8d': 56, '49ff': 57, 'ba16': 58, '6279e1e40e63': 59, '25d6c598': 60, '048d': 61, '487b': 62, 'a992': 63, '573824906bc6': 64, '39e60bae': 65, 'e602':

In [11]:
#tokinezer_tags_2.word_index


In [12]:
sequences_text = tokinezer_text.texts_to_sequences(news)
sequences_tags_1 = tokinezer_tags_1.texts_to_sequences(tags_1)

In [13]:
index = 1
print(sequences_text[:1])
print(sequences_tags_1[:10])

[[1627, 111, 3140, 759, 167, 8, 1398]]
[[1], [1], [3], [1], [11], [3], [1], [1], [1], [1]]


In [14]:
df['clean_tags_1'] = sequences_tags_1

In [109]:
df.head(100)

,date,timestamp,title,level2,level3,clean_tags_1
0,2012/10/01,1349064000000,"Catchings, January help Fever even series with...",sports,wnba,[1]
1,2012/10/01,1349064000000,Kyle Busch rants on radio after his Toyota fal...,sports,nascar,[1]
2,2012/10/01,1349064000000,Schwarzenegger says 'You can't run from your m...,life,books,[3]
3,2012/10/01,1349064000000,Ryder Cup Day 3,sports,golf,[1]
4,2012/10/01,1349064000000,Regular officials blow another big call agains...,gameon,0,[11]
...,...,...,...,...,...,...
95,2012/10/01,1349064000000,Colts coach Chuck Pagano battling leukemia,sports,nfl,[1]
96,2012/10/01,1349064000000,"DiCaprio, Maguire, others urge voter expression",life,people,[3]
97,2012/10/01,1349064000000,Will there be an 'October Surprise' in Electio...,theoval,0,[13]
98,2012/10/01,1349064000000,Country star Jason Aldean apologizes to fans,entertainment,0,[8]


In [16]:
x_train = pad_sequences(sequences_text)

In [17]:
x_train[:15]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        1627,  111, 3140,  759,  167,    8, 1398],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0, 1109, 1677,
           6, 2495,   13,   62, 2343,  790,  564],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        6330,   37, 5013,  575,  149,   14,   56],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 2466,   69,   46,   22],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0, 2551,  852, 2208,  232,
          57,  409,   82,    4,  512,  961,  592],
     

In [18]:
#y_train = pad_sequences(sequences_tags_1)
df = df.fillna(0)
#y_train = utils.to_categorical()
#print(i for i in df['clean_tags_1'])



In [19]:
#y_train = utils.to_categorical(list(map(lambda x: x[0],df['clean_tags_1'])))
row = []
for i in df['clean_tags_1']:
    if i != []:
        row.append(i[0])
    else:
        row.append(0)
print(row[:20])

[1, 1, 3, 1, 11, 3, 1, 1, 1, 1, 2, 3, 1, 5, 4, 1, 2, 5, 3, 1]


In [20]:
y_train = utils.to_categorical(row, 66)

In [21]:
y_train[:50]

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [22]:
model_cnn = Sequential()
model_cnn.add(layers.Embedding(10000, 32, input_length=29))
model_cnn.add(layers.Conv1D(250, 5, padding='valid', activation='relu'))
model_cnn.add(layers.GlobalMaxPooling1D())
model_cnn.add(layers.Dense(130, activation='relu'))
model_cnn.add(layers.Dense(66, activation='softmax'))

In [23]:
model_cnn.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['acc'])

In [24]:
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 29, 32)            320000    
                                                                 
 conv1d (Conv1D)             (None, 25, 250)           40250     
                                                                 
 global_max_pooling1d (Globa  (None, 250)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 130)               32630     
                                                                 
 dense_1 (Dense)             (None, 66)                8646      
                                                                 
Total params: 401,526
Trainable params: 401,526
Non-trainable params: 0
__________________________________________________

In [25]:
model_cnn_save = 'my_model.h5'
check_point = ModelCheckpoint(model_cnn_save,
                              monitor='val_acc',
                              save_best_only=True,
                              verbose=1,
                              mode='max')


In [493]:
history_cnn = model_cnn.fit(x_train, y_train,
                            epochs=2,
                            batch_size=128,
                            validation_split=0.1,
                            callbacks=[check_point])

Epoch 1/2
14638/14640 [============================>.] - ETA: 0s - loss: 0.5245 - acc: 0.8433
Epoch 1: val_acc improved from -inf to 0.84994, saving model to my_model.h5
14640/14640 [==============================] - 369s 25ms/step - loss: 0.5245 - acc: 0.8433 - val_loss: 0.4580 - val_acc: 0.8499
Epoch 2/2
14640/14640 [==============================] - ETA: 0s - loss: 0.3976 - acc: 0.8743
Epoch 2: val_acc improved from 0.84994 to 0.86399, saving model to my_model.h5
14640/14640 [==============================] - 403s 27ms/step - loss: 0.3976 - acc: 0.8743 - val_loss: 0.4059 - val_acc: 0.8640


In [505]:
model_lstm = Sequential()
model_lstm.add(Embedding(10000, 128, input_length=50))

model_lstm.add(LSTM(16))
model_lstm.add(Dense(66, activation='sigmoid'))


In [506]:
model_lstm.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['acc'])

In [507]:
model_lstm.summary()

Model: "sequential_52"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_51 (Embedding)    (None, 29, 32)            320000    
                                                                 
 lstm_3 (LSTM)               (None, 16)                3136      
                                                                 
 dense_99 (Dense)            (None, 66)                1122      
                                                                 
Total params: 324,258
Trainable params: 324,258
Non-trainable params: 0
_________________________________________________________________


In [508]:
model_lstm.summary()
model_lstm_save = 'my_model.h5'
check_point_1 = ModelCheckpoint(model_lstm_save,
                              monitor='val_accuracy',
                              save_best_only=True,
                              verbose=1)

Model: "sequential_52"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_51 (Embedding)    (None, 29, 32)            320000    
                                                                 
 lstm_3 (LSTM)               (None, 16)                3136      
                                                                 
 dense_99 (Dense)            (None, 66)                1122      
                                                                 
Total params: 324,258
Trainable params: 324,258
Non-trainable params: 0
_________________________________________________________________


In [509]:
history_lstm = model_cnn.fit(x_train, y_train,
                            epochs=5,
                            batch_size=128,
                            validation_split=0.1,
                            callbacks=[check_point_1])

Epoch 1/5
 7334/14640 [==============>...............] - ETA: 4:42 - loss: 0.3486 - acc: 0.8875

KeyboardInterrupt: 

In [26]:
model_cnn.load_weights(model_cnn_save)

In [225]:
comment = "On 'Sons of Anarchy,' new faces and new fears"
sequence = tokinezer_text.texts_to_sequences([comment])
sequence

[[6, 5, 97, 11, 406, 10, 11, 2319]]

In [226]:
data = pad_sequences(sequence, maxlen=29)

In [227]:
data

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    6,
           5,   97,   11,  406,   10,   11, 2319]], dtype=int32)

In [228]:
result = model_cnn.predict(data)

1/1 [==============================] - 0s 33ms/step


In [229]:
rez = tokinezer_tags_1.word_index

In [230]:
n = 0
for i in result[0]:

    if i == result.max():
        print(i)
        print(list(rez.items())[n-1][0])
    n += 1


0.41763198
life
